In [14]:
# Block 1: importing necessary modules and checking location
import csv
import numpy as np # linear algebra
import pandas as pd # data processing
import sklearn
from matplotlib import pyplot as plt
%matplotlib inline
import pylab as pl
import matplotlib.mlab as mlab
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
%pwd

'C:\\Users\\iyer\\Documents\\KaggleCompetition\\SantanderProduct'

In [4]:
# Block 2: reading from the file

#test = pd.read_csv('/home/kborozdin/data/track/test_ver2.csv')
test = pd.read_csv('test_ver2.csv')
print("length of test:", len(test))

length of test: 929615


C:\Users\iyer\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Block 3: filling null fields with dummy values

fields = test.columns.values.tolist()
print(fields)

null_fields = ['sexo','ult_fec_cli_1t','indrel_1mes','tiprel_1mes',
               'conyuemp','canal_entrada','cod_prov','nomprov','segmento']
#for i in null_fields:
#    print(i, test[i].unique())
null_dummies = {'sexo':'D', 'ult_fec_cli_1t':'2016-06-30', 'indrel_1mes':2., 'tiprel_1mes':'D', 
                'conyuemp':'D','canal_entrada':'DUM','cod_prov':0.,'nomprov':'DUMMY','segmento':'00 - DUMMY'}
#print([null_dummies[i] for i in null_fields])

for fi in null_fields:
    nulls=test.index[test[fi].isnull()]
    print(len(nulls), fi, '<---', null_dummies[fi])
    test[fi][nulls] = null_dummies[fi]

['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento']
5 sexo <--- D
927932 ult_fec_cli_1t <--- 2016-06-30


C:\Users\iyer\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


23 indrel_1mes <--- 2.0
23 tiprel_1mes <--- D
929511 conyuemp <--- D
2081 canal_entrada <--- DUM
3996 cod_prov <--- 0.0
3996 nomprov <--- DUMMY
2248 segmento <--- 00 - DUMMY


In [6]:
# Block 4: checking for remaining nulls - should be all zeroes at this point

print(test.isnull().sum().sum())

0


In [7]:
# Block 5: started conversion of all fields to integers

test.age=np.rint(test.age)

for fi in ["fecha_dato", "fecha_alta"]:
    test[fi] = pd.to_datetime(test[fi],format="%Y-%m-%d")
    print(fi, min(test[fi]), max(test[fi]))

ind_dato = pd.DatetimeIndex(test["fecha_dato"])
ind_alta = pd.DatetimeIndex(test["fecha_alta"])
test["alta"]=ind_dato.month - ind_alta.month + 12.* (ind_dato.year - ind_alta.year)
print('alta:', min(test.alta), max(test.alta))

test["ult_fec"]=1
test["ult_fec"][test.ult_fec_cli_1t < max(test.ult_fec_cli_1t)]=0
print('ult_fec:', min(test.ult_fec), max(test.ult_fec))

del ind_dato
del ind_alta

fecha_dato 2016-06-28 00:00:00 2016-06-28 00:00:00
fecha_alta 1995-01-16 00:00:00 2016-05-31 00:00:00
alta: 1.0 257.0


C:\Users\iyer\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ult_fec: 0 1


In [8]:
#Block 6: dropping columns we are not going to use

droplist=['tipodom','fecha_dato','ncodpers','fecha_alta','ult_fec_cli_1t']
for fi in droplist:
    test = test.drop(fi, 1)
print(list(test.columns))

['ind_empleado', 'pais_residencia', 'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'cod_prov', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento', 'alta', 'ult_fec']


In [9]:
# Block 7: forming data frames with 'NA' only in renta: norenta
# and without 'NA' rows in renta: terenta 

norenta = test[test.renta == min(test.renta)]
terenta = test[test.renta > min(test.renta)]
print(len(norenta), len(terenta))
terenta.renta=terenta.renta.apply(np.float)

227965 701650


C:\Users\iyer\Anaconda3\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
# Block 8: an important and somewhat controversial step - substituting column values with mostly median renta values

all_fields=['ind_empleado', 'pais_residencia', 'sexo', 'age', 'ind_nuevo', 
            'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi',
            'indext', 'conyuemp', 'canal_entrada', 'indfall', 
            'cod_prov', 'nomprov', 'ind_actividad_cliente', 
            'segmento', 'alta', 'ult_fec']
# In[17]:

for fi in all_fields:
    uniques=test[fi].unique()
    print(uniques)
    for i in uniques:
#        print i, len(test.age[test[fi]==i]), 'age:', test.age[test[fi]==i].mean(), test.age[test[fi]==i].median()
#        print i, len(terenta.renta[terenta[fi]==i]), 'renta:', terenta.renta[terenta[fi]==i].mean(), terenta.renta[terenta[fi]==i].median() 
        if fi != 'nomprov':
            if len(terenta.renta[terenta[fi]==i]) > 0:
                test[fi][test[fi]==i] = np.rint(terenta.renta[terenta[fi]==i].median())
            else:
                test[fi][test[fi]==i] = np.rint(terenta.renta.median())
        else:
            test[fi][test[fi]==i] = np.rint(test.age[test[fi]==i].mean())

    print(test[fi].unique())

['F' 'N' 'A' 'B' 'S']


C:\Users\iyer\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[148348.0 101457.0 143749.0 171414.0 87798.0]
['ES' 'CH' 'DE' 'GB' 'BE' 'DJ' 'IE' 'QA' 'US' 'VE' 'DO' 'SE' 'AR' 'CA' 'PL'
 'CN' 'CM' 'FR' 'AT' 'RO' 'LU' 'PT' 'CL' 'IT' 'MR' 'MX' 'SN' 'BR' 'CO' 'PE'
 'RU' 'LT' 'EE' 'MA' 'HN' 'BG' 'NO' 'GT' 'UA' 'NL' 'GA' 'IL' 'JP' 'EC' 'IN'
 'DZ' 'ET' 'SA' 'HU' 'JM' 'CI' 'CU' 'BO' 'TG' 'TN' 'NG' 'AU' 'GR' 'DK' 'LB'
 'UY' 'TH' 'SG' 'MD' 'SK' 'AD' 'BY' 'HK' 'HR' 'EG' 'GQ' 'PR' 'ZA' 'PA' 'KE'
 'TR' 'FI' 'BA' 'SV' 'PY' 'PK' 'KR' 'AO' 'GN' 'IS' 'TW' 'MK' 'VN' 'CZ' 'CR'
 'MZ' 'MT' 'LY' 'GH' 'KH' 'AE' 'RS' 'OM' 'GE' 'NI' 'GI' 'NZ' 'MM' 'PH' 'KW'
 'BM' 'CG' 'ML' 'AL' 'ZW' 'CF' 'GM' 'CD' 'BZ' 'KZ' 'GW' 'SL' 'LV']


C:\Users\iyer\Anaconda3\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[101487.0 132336.0 167288.0 111760.0 60424.0 101491.0 130474.0 266374.0
 143137.0 227980.0 114949.0 138554.0 719776.0 172233.0 409550.0 242701.0
 116770.0 73244.0 113993.0 160881.0 385514.0 185340.0 187271.0 128108.0
 125731.0 142913.0]
['V' 'H' 'D']
[103335.0 99365.0 104297.0]
[  56.   36.   22.   51.   41.   33.   23.   43.   63.   62.   32.   58.
   71.   31.   30.   59.   45.   37.   39.   38.   46.   34.   35.   42.
   29.   88.   64.   48.   72.   47.   27.   24.   50.   49.   57.   67.
   25.   28.   13.   40.   53.   54.   26.   11.   21.   60.   44.   55.
    7.   52.   66.   90.   73.   78.   79.   61.   69.   12.    9.   65.
   77.   83.   81.   10.    5.   18.   17.   87.   70.   80.   75.   68.
   74.   16.    4.   82.   91.   76.   19.   15.    6.   97.   89.   85.
   86.   14.   20.    8.   84.   95.   93.  100.   96.   92.   98.   94.
  105.  102.  101.  104.  103.   99.    3.    2.  116.  106.  107.  109.
  110.  117.  108.  113.  111.  112.  114.  164.  118.  127.]
[ 

C:\Users\iyer\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[106034.0 100585.0 96920.0 101869.0 102161.0 99847.0 98472.0 98414.0
 95346.0 95656.0 100425.0 101785.0 98409.0 99990.0 101391.0 98708.0 94778.0
 95002.0 99215.0 99837.0 97035.0 102628.0 94674.0 107048.0 100963.0
 102093.0 98949.0 96042.0 95256.0 99944.0 104473.0 100303.0 97181.0
 102693.0 97185.0 103520.0 108038.0 100512.0 100632.0 96453.0 100616.0
 95530.0 96100.0 101679.0 98539.0 97711.0 102297.0 106460.0 96153.0
 105758.0 107487.0 109876.0 105649.0]
[1 0]
[108244  96474]
['01 - TOP' '02 - PARTICULARES' '03 - UNIVERSITARIO' '00 - DUMMY']
[140760.0 107028.0 89309.0 127992.0]
[ 257.   34.    5.   33.   32.   15.   31.   27.   20.    2.   18.   30.
    4.   29.   19.   26.   24.   28.   95.    7.    8.   21.    9.   22.
   25.    6.   16.   12.    1.   14.    3.   10.   17.   13.   37.   36.
   11.   23.   35.   38.   42.  150.   54.  225.   41.   40.  120.   39.
  100.  138.  110.   56.  169.  170.  155.  159.  132.  137.  115.   47.
   48.   46.   45.   44.   49.   43.   51.   50.   

In [11]:
# Block 9: need to repeat block 7
# forming data frames with 'NA' only in renta: norenta
# and without 'NA' rows in renta: terenta 

norenta = test[test.renta == min(test.renta)]
terenta = test[test.renta > min(test.renta)]
print(len(norenta), len(terenta))
terenta.renta=terenta.renta.str.replace(' ','')
terenta.renta=terenta.renta.astype('float')
terenta.renta=terenta.renta.apply(np.float)

227965 701650


C:\Users\iyer\Anaconda3\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [12]:
# Block 10: unnecessary - just looking at what we got so far

for i in list(terenta.columns):
    if i != 'renta':
        terenta[i]=terenta[i].astype('int')        
    print(i, min(terenta[i]), max(terenta[i]))

print(list(terenta.columns))
terenta.corr()

C:\Users\iyer\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ind_empleado 87798 171414
pais_residencia 60424 719776
sexo 99365 104297
age 87139 221708
ind_nuevo 98900 101493
antiguedad 80753 233576
indrel 100398 101493
indrel_1mes 94975 101491
tiprel_1mes 94975 109296
indresi 101487 142913
indext 87745 102134
conyuemp 101482 239327
canal_entrada 57291 210914
indfall 101433 128167
cod_prov 62260 156021
nomprov 94674 109876
ind_actividad_cliente 96474 108244
renta 1202.73 28894395.51
segmento 89309 140760
alta 82774 240017
ult_fec 100398 101493
['ind_empleado', 'pais_residencia', 'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'cod_prov', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento', 'alta', 'ult_fec']


,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,...,conyuemp,canal_entrada,indfall,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,alta,ult_fec
ind_empleado,1.000000,-0.000121,-0.004837,0.015849,0.000620,0.079308,0.000832,0.000074,0.021378,-0.000211,...,0.325911,0.021727,-0.000098,0.023981,0.025319,0.017470,0.005590,0.027253,0.085130,0.000832
pais_residencia,-0.000121,1.000000,0.001315,0.003808,0.000117,0.005252,0.000157,0.000014,0.004501,0.571792,...,-0.000056,0.003697,-0.000258,0.007796,0.007072,0.004466,0.004828,0.006086,0.005544,0.000157
sexo,-0.004837,0.001315,1.000000,0.128374,-0.003467,0.135580,-0.001472,-0.000704,0.060165,0.002470,...,0.001027,0.099024,0.013945,0.025100,0.035306,0.046784,0.006916,0.135826,0.148024,-0.001472
age,0.015849,0.003808,0.128374,1.000000,-0.000516,0.635334,-0.000495,0.001283,0.188919,0.001569,...,0.014086,0.562776,0.067441,0.239075,0.288233,0.150424,0.069632,0.633460,0.639104,-0.000495
ind_nuevo,0.000620,0.000117,-0.003467,-0.000516,1.000000,0.006158,0.008011,0.099207,-0.016382,0.000205,...,0.000289,0.010099,0.001326,-0.003311,-0.004277,-0.012729,0.001085,-0.018728,0.007842,0.008011
antiguedad,0.079308,0.005252,0.135580,0.635334,0.006158,1.000000,0.010145,0.000841,0.186104,0.004677,...,0.089782,0.595499,0.052992,0.301847,0.339202,0.139197,0.084989,0.557346,0.958556,0.010145
indrel,0.000832,0.000157,-0.001472,-0.000495,0.008011,0.010145,1.000000,-0.000096,0.002167,0.000275,...,0.000387,0.003155,-0.025570,-0.001574,-0.001897,0.021699,0.001236,-0.003667,0.009547,1.000000
indrel_1mes,0.000074,0.000014,-0.000704,0.001283,0.099207,0.000841,-0.000096,1.000000,0.002907,0.000024,...,0.000034,-0.003655,0.000158,-0.001115,-0.000997,-0.000406,0.000517,-0.004695,0.000895,-0.000096
tiprel_1mes,0.021378,0.004501,0.060165,0.188919,-0.016382,0.186104,0.002167,0.002907,1.000000,0.007936,...,0.010382,0.216687,0.015366,0.125774,0.145522,0.838484,0.029345,0.324759,0.193799,0.002167
indresi,-0.000211,0.571792,0.002470,0.001569,0.000205,0.004677,0.000275,0.000024,0.007936,1.000000,...,-0.000098,0.004444,-0.000451,0.013491,0.012275,0.007875,0.002311,0.012516,0.004946,0.000275


In [41]:
# Block 11: do linear regression of the model and print the results

import statsmodels.api as sm

x = terenta.drop('renta', 1)
#x = sm.add_constant(x)
x = np.asarray(x)
#print(x)
y = np.log10(terenta.renta)
y = np.asarray(y)
linear = sm.OLS(y,x)
ols_fit = sm.OLS(y,x).fit()
print(ols_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.538e+07
Date:                Wed, 23 Nov 2016   Prob (F-statistic):               0.00
Time:                        13:20:17   Log-Likelihood:                -11937.
No. Observations:              701650   AIC:                         2.391e+04
Df Residuals:                  701631   BIC:                         2.413e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1         -1.423e-07   2.27e-07     -0.627      0.5

In [42]:
from sklearn import linear_model
#print(x)
regr = linear_model.LinearRegression()
regr.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [43]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x) - y) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Coefficients: 
 [ -1.64098821e-07   1.09688451e-06   9.26104943e-08   1.07992491e-06
   3.05544126e-06   4.78876805e-07   2.69288736e-06   1.14765850e-05
   6.82570519e-07  -2.86981409e-06   4.68480865e-06   4.81465589e-07
   3.33663662e-07   8.29197517e-07   4.32536101e-06  -3.24273350e-06
   3.83516801e-07  -8.11864685e-07   7.15688721e-07   2.69288736e-06]
Mean squared error: 0.06
Variance score: 0.21
